# npr MC1: Cleantech Retrieval Augemented Generation

**Dominik Filliger, Nils Fahrni, Noah Leuenberger**

> The topic of Mini-Challenge 1 is retrieval augmented generation (RAG) incorporating a combination of unsupervised learning, pre-training and in-context learning techniques.

- [Description of the task](https://spaces.technik.fhnw.ch/storage/uploads/spaces/81/exercises/NPR-Mini-Challenge-1-Cleantech-RAG-1708982891.pdf)
- [Introduction to RAG](https://spaces.technik.fhnw.ch/storage/uploads/spaces/81/exercises/Retrieval-Augmented-Generation-Intro-1709021241.pdf)


# Structure of our Solution

This notebook serves as the main entry point for our solution to the NPR Mini-Challenge 1. We will provide a detailed explanation of our approach and the code we used to solve the task. However, we have outsourced the code for the evaluation, Langchain LLM model creation and vectorstore interaction to script files which can be found in the `src` directory. We will reference these scripts in the respective sections and explain the code in place.

Additionally, scripts for the development subset and subset evaluation set creation can be found in the `scripts` directory and will be referenced in their respective sections. These were primarily used to speed up the development process and are not necessary for the final solution as we will use the full dataset for the latest evaluation.

The notebook starts by setting up a baseline pipeline. After that, we will conduct a series of experiments to explore the impact of different strategies on the performance of the pipeline. The experiments are designed to test the effectiveness of different retrieval and generation strategies in the RAG pipeline.



# Setup

The setup section is used to import the necessary libraries and set the configuration for the notebook. We will also load the environment variables from the `.env` file to set the configuration for the notebook. The configuration includes the paths to the data files and the configuration for the Phoenix library.

There is an option of three boolean variables which can be set to `True` or `False` to configure the notebook:
- `DEV_MODE`: If set to `True`, the notebook will use the development subset of the data. If set to `False`, the notebook will use the full dataset.
- `USE_GPU`: If set to `True`, the notebook will use the GPU for the computations. If set to `False`, the notebook will use the CPU.
- `USE_CACHE`: If set to `True`, the notebook will use the cache for the computations. If set to `False`, the notebook will not use the cache.

The caching is especially useful when running the notebook multiple times to avoid recomputing the same results. Especially the evaluation of the pipelines can be time-consuming and require a lot of tokens from the Azure API. Therefore, it is recommended to set `USE_CACHE` to `True` when running the notebook multiple times.

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

DEV_MODE = False
USE_GPU = False
USE_CACHE = True

## Pathing

The pathing section is used to set the paths to the data files. The paths are set based on the `DEV_MODE` variable. If `DEV_MODE` is set to `True`, the paths to the development subset of the data are used. If `DEV_MODE` is set to `False`, the paths to the full dataset are used. The paths are then used to load the data files into pandas DataFrames. This is sole for convenience and to have a clear separation of the paths in one place.

In [ ]:
def get_path(dev_mode=DEV_MODE):
    if dev_mode:
        # Paths used in development mode
        return {
            'df_path': 'data/subset/cleantech_media_dataset_v2_2024-02-23_subset.csv',
            'df_eval_path': 'data/subset/cleantech_media_dataset_v2_2024-02-23_subset_eval.csv',
            'df_preprocessed_path': get_preprocessed_path(
                'data/subset/cleantech_media_dataset_v2_2024-02-23_subset.csv'),
            'df_eval_preprocessed_path': get_preprocessed_path(
                'data/subset/cleantech_media_dataset_v2_2024-02-23_subset_eval.csv')
        }
    else:
        # Paths used in production mode
        return {
            'df_path': 'data/Cleantech Media Dataset/cleantech_media_dataset_v2_2024-02-23.csv',
            'df_eval_path': 'data/Cleantech Media Dataset/cleantech_rag_evaluation_data_2024-02-23.csv',
            'df_preprocessed_path': get_preprocessed_path(
                'data/Cleantech Media Dataset/cleantech_media_dataset_v2_2024-02-23.csv'),
            'df_eval_preprocessed_path': get_preprocessed_path(
                'data/Cleantech Media Dataset/cleantech_rag_evaluation_data_2024-02-23.csv')
        }


def get_preprocessed_path(path: str) -> str:
    return path.replace(".csv", "_preprocessed.csv")


paths = get_path()
print(paths)

## Observability & Monitoring

> Phoenix is an open-source observability library designed for experimentation, evaluation, and troubleshooting. It allows AI Engineers and Data Scientists to quickly visualize their data, evaluate performance, track down issues, and export data to improve.

We will use Phoenix to visualize traces to quickly debug pipelines. The library offers way more feature which we will not use. Down below we add the Phoenix callbacks to Langchain, our main library for the solution, to visualize the traces.

To get quick access to the Phoenix dashboard, open the following link in your browser: [http://localhost:6006](http://localhost:6006)


In [ ]:
from phoenix.trace.langchain import LangChainInstrumentor
import phoenix as px

try:
    px.close_app()
    session = px.launch_app()
except:
    print(
        "Could not launch Phoenix app. Please make sure any existing Phoenix app process is closed. Restart the kernel if necessary.")

LangChainInstrumentor().instrument()

# Data Loading & Preprocessing

Next we will load the data and preprocess it. The data is loaded into a pandas DataFrame and then preprocessed. This step is based on the findings and explorations made in the EDA notebook (`notebooks/exploration.ipynb`). 

The dataset used in this Mini-Challenge is the Cleantech Media Dataset, which contains articles about clean technology and sustainability topics. The dataset consists of various columns, including the URL, domain, title, author, date, and content of the articles. The content column is of main interest as it contains the text data which the RAG piepline will be built upon. The dataset can be found [on Kaggle](https://www.kaggle.com/datasets/jannalipenkova/cleantech-media-dataset) together with an evaluation set which will be discussed later in the notebook.

The preprocessing code itself can be found in the `src/preprocessing.py` file. The code is encapsulated in a class called `Preprocessor` which is used to preprocess the data which will be indexed and used for the retrieval augmented generation pipeline. There is also a separate class called `EvaluationPreprocessor` which is used to preprocess the evaluation set which will be discussed later in the notebook.

The preprocessing includes the following steps:

1. **Language Detection and Filtering:** Detect the language of each text chunk and filter out non-English chunks to maintain language consistency in the dataset.
2. **HTML Cleaning:** Remove any HTML tags from the text content to ensure that only relevant textual data is processed.
3. **Removing Special Characters:** Clean up the text by removing non-alphanumeric characters, keeping only letters, numbers, and necessary punctuation.
4. **Duplicate Removal:** Identify and remove duplicate text chunks based on content to ensure uniqueness in the dataset.
5. **Content Concatenation:** If the text content is split across multiple entries or rows, concatenate these into a single text chunk per dataset entry.
6. **Adding Unique Identifiers:** Generate and append a unique identifier to each row based on the content's hash value, which helps in tracking and indexing the data for the evaluation process of the retrieval.

In [ ]:
import pandas as pd

df = pd.read_csv(get_path()['df_path'])
df.head()

The preprocessed data is then saved to a new CSV file to avoid recomputing the preprocessing steps when running the notebook multiple times.

In [ ]:
from src.preprocessing import Preprocessor

if os.path.exists(paths['df_preprocessed_path']) and USE_CACHE:
    df = pd.read_csv(paths['df_preprocessed_path'])
else:
    df = Preprocessor(df).preprocess()
    df.to_csv(paths['df_preprocessed_path'], index=False)

# Indexing

The indexing step is used to embed the documents and store them in a vector store. The vector store is used to store the embeddings and provide an interface to interact with the embeddings.

## Chunking

Another important step in the indexing process is to split the documents into smaller chunks. This division is crucial for managing large documents, improving the granularity of search results, and enhancing the efficiency of the embedding process.

**Purpose and Benefits**:

- **Manageability**: Smaller chunks are easier to process and store, reducing the computational burden on the system.
- **Precision**: By embedding smaller sections of text, the system can achieve more precise document retrieval, particularly for queries that target specific information within a large document.
- **Performance**: Smaller text chunks lead to faster processing times during both embedding and retrieval phases.

For the baseline pipeline, we will use the Recursive Character Text Splitter from the Langchain library. This splitter divides the text into smaller chunks based on the character count, ensuring that each chunk is of a manageable size for processing and embedding. The following parameters are used for the splitter:

- **Chunk Size**: The maximum number of characters in each chunk.
    - We use a chunk size of 1000 characters to ensure that the chunks are small enough for efficient processing but large enough to capture meaningful information. 
- **Chunk Overlap**: The number of characters that overlap between adjacent chunks.
    - We set the overlap to 0 to avoid redundancy and ensure that each chunk contains unique information.   
- **Length Function**: The function used to calculate the length of the text.
- **Is Separator Regex**: A flag indicating whether the separator is a regular expression.

The recursive character text splitter is a simple and effective method for dividing text into manageable chunks, making it suitable for our initial experiments with the RAG pipeline. 

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)

The helper function `create_documents` is used to create the documents from the DataFrame. The function takes the DataFrame and the text splitter as input and returns the documents. The function also prints the number of documents created and the percentage of documents created compared to the number of rows in the source DataFrame.

In [ ]:
def create_documents(df: pd.DataFrame, text_splitter, verbose=True):
    metadata_cols = ['url', 'domain', 'title', 'author', 'date', 'id']
    if not all(col in df.columns for col in metadata_cols + ['content']):
        raise ValueError(
            f"DataFrame must contain all metadata columns and a 'content' column: {metadata_cols + ['content']}")

    metadata = df[metadata_cols].rename(columns={'id': 'origin_doc_id'}).to_dict('records')
    docs = text_splitter.create_documents(df['content'], metadata)

    if verbose:
        print(
            f"{text_splitter.__class__.__name__}: "
            f"Number of documents created: {len(docs)}, "
            f"Number of rows in source df: {len(df)}, "
            f"Percentage of documents created: {len(docs) / len(df) * 100:.2f}%")

    return docs


documents = create_documents(df, recursive_text_splitter)

In [ ]:
documents[0]

Once split, each text chunk is processed through the BGE embeddings model to generate individual vectors. These vectors are then stored in the vector store. Handling multiple embeddings per document involves indexing each chunk separately while maintaining a reference to the original document.

## Embeddings

We will use the [BGE embeddings from Hugging Face](https://huggingface.co/BAAI/bge-small-en) to embed the documents. BGE embeddings are transformer-based models trained on a vast corpus of text data, making them highly effective for a variety of NLP tasks. The embeddings convert text data into dense vectors of fixed dimensionality, typically via processes including tokenization and normalization. The resulting vectors capture the semantic properties of the input texts.

**Why specifically BGE embeddings?**

- **High Semantic Fidelity**: BGE embeddings are designed to capture deep semantic meanings, enabling more effective document similarity checks and retrieval.
- **Performance**: They are optimized for both accuracy and speed in large-scale applications.
- **Versatility**: Suitable for diverse NLP tasks such as classification, clustering, and information retrieval.
- **Accessibility through Langchain**: The Langchain library provides a simple interface to interact with the BGE embeddings, making it easier to integrate them into the RAG pipeline.

In [ ]:
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu" if not USE_GPU else "cuda:0"},
    encode_kwargs={"normalize_embeddings": True}
)

## Vector Store

We will use [ChromaDB](https://www.trychroma.com/) to store the embeddings. For easier interaction with the embeddings, we will use the `VectorStore` class which can be found in the `src/vector_store.py` file. The `VectorStore` class provides an interface that combines a specific collection of embeddings with the Langchain VectorStore API. The class allows for easy interaction with the embeddings, including adding documents and performing similarity searches.

The embedding of the chunks takes time and resources, so we will use a persistent store to save the embeddings and avoid recomputing them when running the notebook multiple times. A database file with the precomputed embeddings can be downloaded and integrated according to the instructions in the README file.

The collection name for the vector store is generated based on the embeddings model and the text splitter used. The collection name is used to identify the specific collection of embeddings in the vector store.

In [ ]:
from src.vector_store import VectorStore


def get_vector_store_collection_name(embeddings, text_splitter):
    return f"cleantech-{embeddings.model_name}-{text_splitter.__class__.__name__}_{'dev' if DEV_MODE else 'prod'}".lower().replace(
        " ", "_").replace("/", "_").replace(":", "_").replace("-", "_")


bge_vector_store = VectorStore(embedding_function=bge_embeddings,
                               collection=get_vector_store_collection_name(bge_embeddings, recursive_text_splitter))

get_vector_store_collection_name(bge_embeddings, recursive_text_splitter)

In the next step we will add the prepared documents from the previous step to the VectorStore. If the collection already exists in the VectorStore, this step is skipped.

In [ ]:
bge_vector_store.add_documents(documents, verbose=True)

After adding the documents to the vector store we can now perform similarity searches on the documents to verify that the interaction with the vector store works as expected.

In [ ]:
bge_vector_store.similarity_search_w_scores("The company is also aiming to reduce gas flaring?")

The vector store returns a list of documents with their respective similarity scores. The documents are sorted by their similarity scores in descending order. The similarity scores indicate how similar the documents are to the query text. The higher the similarity score, the more similar the document is to the query text.

# Baseline Pipeline

The baseline pipeline is a first simple implementation of the RAG pipeline. As the name suggests, it serves as the starting point for our experiments and provides a reference for comparison with the subsequent pipelines. The baseline pipeline consists of the following components:

- **Embeddings**: We use the BGE embeddings from Hugging Face to embed the documents.
- **Retriever**: In the baseline pipeline, we use the default vector retriever from ChromaDB. It returns the top-4 most similar documents based on the cosine similarity.
- **LLM Model**: We use the GPT-3.5 model provided on Azure as our LLM model. The model is responsible for generating the answer to the question based on the retrieved documents. The temperature parameter is set to the default of 0.7. 

In [ ]:
from src.generation import get_llm_model, LLMModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

base_retriever = bge_vector_store.get_retriever()
azure_model = get_llm_model(LLMModel.GPT_3_AZURE)

We will use the following prompt template to generate the chat prompt for the LLM model. The template includes the context and the question to provide the necessary information for the model to generate the answer and is fairly simple.

In [ ]:
base_rag_prompt = """
Answer the question to your best knowledge when looking at the following context:
{context}
                
Question: {question}
"""

The `base_rag_chain` is the core of the baseline pipeline. It consists of the following steps:

1. **Context Formatting**: The context documents are formatted into a single string to provide the necessary information to the LLM model.
2. **Chat Prompt Generation**: The chat prompt is generated using the template defined above.
3. **LLM Model Invocation**: The LLM model is invoked to generate the answer based on the context and the question.
4. **Output Parsing**: The output from the LLM model is parsed to extract the answer from the response.
5. **Answer Assignment**: The answer is assigned to the runnable parallel chain where it is combined with the context and the question. Context and question are also returned for further reference in the evaluation.

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


base_rag_chain = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | ChatPromptTemplate.from_template(base_rag_prompt)
        | azure_model
        | StrOutputParser()
)

base_rag = RunnableParallel(
    {
        "context": base_retriever,
        "question": RunnablePassthrough()
    }
).assign(answer=base_rag_chain)

The `base_rag` pipeline/chain can now be used to generate answers for questions. We will test the pipeline with a sample question to verify that it works as expected.

In [ ]:
base_rag.invoke("Is the company aiming to reduce gas flaring?")

# Evaluation Setup

In order to compare the performance of different pipelines we need to evaluate them. The evaluation is done with the `ragas` library and classical non-LLM based metrics. 

`ragas` provides predefined metrics for the evaluation which are described in the [documentation](https://docs.ragas.io/en/stable/concepts/metrics/index.html). 

Further an overview of the metrics used in the evaluation is provided below:

## RAGAS Metrics

The `ragas` metrics are LLM-based evaluated metrics where the LLM is used to rate the generated answers and retrieved contexts against labeled ground truths. As LLM are known for their non-deterministic behavior, the metrics should be taken with a grain of salt. Nevertheless, they provide a relatively simple and straightforward way to evaluate the performance.

1. [**Answer Correctness**](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html): This metric assesses the accuracy of the generated answer compared to the ground truth. It combines semantic and factual similarities between the answer and ground truth, providing a score ranging from 0 to 1. A higher score indicates that the generated answer closely aligns with the ground truth, reflecting a more correct response.

2. [**Context Precision**](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html): This metric evaluates the relevance of the contexts retrieved by the RAG system in relation to the ground truth. It checks if the relevant items from the ground truth are ranked higher in the retrieved contexts. Scores range from 0 to 1, where higher values suggest better precision and relevance of the retrieved information.

3. [**Answer Relevancy**](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html): Focused on the pertinence of the generated answer to the question, this metric scores based on how complete and free from redundant information the answer is. Higher scores indicate more relevant answers. It uses the mean cosine similarity between the original question and artificial questions generated based on the answer.

4. [**Answer Semantic Similarity**](https://docs.ragas.io/en/stable/concepts/metrics/semantic_similarity.html): This metric evaluates the semantic resemblance between the generated answer and the ground truth. It uses a cross-encoder model to calculate semantic similarity, with scores between 0 and 1. Higher scores signify a better semantic alignment between the generated answer and the ground truth.

5. [**Context Entity Recall**](https://docs.ragas.io/en/stable/concepts/metrics/context_entities_recall.html): This metric measures the recall of entities in the retrieved contexts against those in the ground truths. It is particularly useful in scenarios where factual accuracy and entity coverage are crucial, such as in historical QA or tourism help desks. The score is calculated by comparing the number of entities that overlap between the ground truths and contexts with the total number of entities in the ground truths.

## Non-LLM Based Metrics

In addition to the RAGAS metrics, we will also use the following metrics to evaluate the performance of the pipelines. They provide further specific insights into the retrieval performance.

1. **Reciprocal Rank**: The reciprocal rank is a metric that evaluates the ranking of the correct answer in the list of retrieved answers. It is calculated as the reciprocal of the rank of the correct answer. A higher reciprocal rank indicates that the correct answer is ranked higher in the list of retrieved answers.
2. **Hit@K**: The Hit@K metric evaluates whether the correct answer is present in the top K retrieved answers. It is calculated as a binary value, where 1 indicates that the correct answer is present in the top K retrieved answers, and 0 indicates otherwise. A higher Hit@K value indicates that the correct answer is more likely to be found in the top K retrieved answers.

## Evaluation Data

The evaluation data is part of the datasets provided by the Mini-Challenge task [from Kaggle](https://www.kaggle.com/datasets/jannalipenkova/cleantech-media-dataset). The data consists of questions and a relevant chunk for each question.

// TODO: Add more information about the evaluation preprocessing 

In [ ]:
df_eval = pd.read_csv(paths['df_eval_path'])
df_eval.head()

In [ ]:
from src.preprocessing import EvaluationPreprocessor

if os.path.exists(paths['df_eval_preprocessed_path']) and USE_CACHE:
    df_eval = pd.read_csv(paths['df_eval_preprocessed_path'])
else:
    df_eval = EvaluationPreprocessor(df, df_eval).preprocess()
    df_eval.to_csv(paths['df_eval_preprocessed_path'], index=False)

df_eval.head()

In [ ]:
HANDPICKED_QUESTIONS = [
    "Why does melting ice contribute to global warming?",
    "In 2021, what were the top 3 states in the US in terms of total solar power generating capacity?",
    "What is the EU’s Green Deal Industrial Plan?"
]


def ask_handpicked_questions(rag_chain, questions=HANDPICKED_QUESTIONS):
    answers = {question: rag_chain.invoke(question) for question in questions}

    for question, answer in answers.items():
        print(f"Question: {question}")
        print(f"Answer: {answer['answer']}")
        print("\n")


ask_handpicked_questions(base_rag)

In [ ]:
from src.evaluation import Evaluator

base_evaluator = Evaluator(name="Baseline",
                           cache_results=USE_CACHE,
                           rag_chain=base_rag,
                           llm_model=azure_model,
                           embeddings=bge_embeddings)

base_evaluator.evaluate(df_eval)
base_evaluator.plot_summary()

In [ ]:
base_evaluator.get_summary()

# Experiment 1: Looking at the impact of context and its chunking strategy

Contrary to the apparent structure of the data, which seems to have already chunked the data according to recursive character chunking strategy, this step will introduce Semantic Chunking. This will help us to see if it is beneficial for the LLM to consider the context in a more structured manner.

In order to embed the processed documents we again can turn them into langchain-digestible Documents.

In [ ]:
%%script false --no-raise-error

from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    bge_embeddings, breakpoint_threshold_type="percentile"
)

semantic_documents = create_documents(df, semantic_chunker)
semantic_documents[0]

And in order to look at this experiment in an encapsulated manner, a new `VectorStore` will be created with a separate collection to keep concerns separated.

In [ ]:
%%script false --no-raise-error

bge_semantic_vector_store = VectorStore(embedding_function=bge_embeddings,
                                        collection=get_vector_store_collection_name(bge_embeddings, semantic_chunker))

semantic_retriever = bge_semantic_vector_store.get_retriever()

In [ ]:
%%script false --no-raise-error

bge_semantic_vector_store.add_documents(semantic_documents, verbose=True, batch_size=128)

In [ ]:
%%script false --no-raise-error

bge_semantic_vector_store.similarity_search_w_scores("The company is also aiming to reduce gas flaring?")

In [ ]:
%%script false --no-raise-error

semantic_rag = RunnableParallel(
    {
        "context": semantic_retriever,
        "question": RunnablePassthrough()
    }
).assign(answer=base_rag_chain)

In [ ]:
%%script false --no-raise-error

semantic_rag.invoke("Is the company aiming to reduce gas flaring?")

### Results

In [ ]:
%%script false --no-raise-error

semantic_evaluator = Evaluator(name="Semantic Chunking",
                               cache_results=USE_CACHE,
                               rag_chain=semantic_rag,
                               llm_model=azure_model,
                               embeddings=bge_embeddings)

semantic_evaluator.evaluate(df_eval)
semantic_evaluator.plot_summary()

In [ ]:
%%script false --no-raise-error

semantic_evaluator.get_summary()

In [ ]:
%%script false --no-raise-error

ask_handpicked_questions(semantic_rag)

### Conclusion

# Experiment 2: Using a Multi-Query Retrieval Strategy

At the heart of the RAG is the retriever, which is responsible for finding the most relevant documents for a given question. The baseline RAG uses the vector retriever to find the most relevant document, using cosine-similarity. 

We will now experiment with a multi-query retrieval strategy. The idea is to use multiple queries to retrieve a multitude of documents and take a unique union of the results. This way we can increase the diversity of the documents and potentially improve the quality of the generated answer. 

For this we will use the MultiQueryRetriever from langchain.


In [ ]:
from langchain.retrievers import MultiQueryRetriever

mqr_retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever, llm=azure_model
)

In [ ]:
## using the langchain template for the prompt
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
        prompt_perspectives
        | azure_model
        | StrOutputParser()
        | (lambda x: x.split("\n"))
)

In [ ]:
from langchain.load import dumps, loads


def get_unique_union(union_docs: list[list]):
    """ Unique union of retrieved docs """
    flattened_docs = [dumps(doc) for sublist in union_docs for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]


mqr_retrieval_chain = (
        generate_queries
        | mqr_retriever.map()
        | get_unique_union
)

In [ ]:
mqr_rag = RunnableParallel(
    {
        "context": mqr_retrieval_chain,
        "question": RunnablePassthrough()
    }
).assign(answer=base_rag_chain)

In [ ]:
mqr_rag.invoke("Is the company aiming to reduce gas flaring?")

### Results

In [ ]:
mqr_evaluator = Evaluator(name="Multi-Query Retrieval",
                          cache_results=USE_CACHE,
                          rag_chain=mqr_rag,
                          llm_model=azure_model,
                          embeddings=bge_embeddings)

mqr_evaluator.evaluate(df_eval)
mqr_evaluator.plot_summary()

In [ ]:
mqr_evaluator.get_summary()

In [ ]:
ask_handpicked_questions(mqr_rag)

### Conclusion

# Experiment 3: Using a Step Back Strategy

In [ ]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        few_shot_prompt,
        ("user", "{question}"),
    ]
)
generate_queries_step_back = (prompt | azure_model | StrOutputParser())

In [ ]:
from langchain_core.runnables import RunnableLambda

response_prompt_template = """
You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:
"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)


def get_unique_documents(stepback_results):
    """
    Combines and filters documents from 'normal_context' and 'step_back_context' to ensure uniqueness.

    This function merges two lists of documents, eliminating any duplicates based on the 'page_content' attribute.
    It assumes that each document in the lists is an object with a 'page_content' attribute that contains the unique content of the document.

    Parameters:
    - stepback_results (dict): A dictionary with two keys 'normal_context' and 'step_back_context'.
      Each key corresponds to a list of document objects with a 'page_content' attribute.

    Returns:
    - list: A list of unique document objects based on their 'page_content'.
    """
    normal_docs = stepback_results['normal_context']
    stepback_docs = stepback_results['step_back_context']

    seen_content = set()
    combined_docs = []
    for doc in normal_docs + stepback_docs:
        if doc.page_content not in seen_content:
            combined_docs.append(doc)
            seen_content.add(doc.page_content)
    return combined_docs


step_back_retrieval_chain = (
    {
        "normal_context": RunnableLambda(lambda x: x) | base_retriever,
        "step_back_context": generate_queries_step_back | base_retriever,
    }
)

step_back_answer_chain = (
        RunnablePassthrough.assign(normal_context=(lambda x: format_docs(x['stepback']["normal_context"])),
                                   step_back_context=(lambda x: format_docs(x['stepback']["step_back_context"])),
                                   question=(lambda x: x["question"]))
        | response_prompt
        | azure_model
        | StrOutputParser()
)
step_back_rag = RunnableParallel(
    {
        "stepback": step_back_retrieval_chain,
        "context": step_back_retrieval_chain | RunnableLambda(lambda x: get_unique_documents(x)),
        "question": RunnablePassthrough()
    }
).assign(answer=step_back_answer_chain)

In [ ]:
step_back_rag.invoke("What is the company's goal in reducing gas flaring?")

### Results

In [ ]:
step_back_evaluator = Evaluator(name="Step Back",
                                cache_results=USE_CACHE,
                                rag_chain=step_back_rag,
                                llm_model=azure_model,
                                embeddings=bge_embeddings)

step_back_evaluator.evaluate(df_eval)
step_back_evaluator.plot_summary()

In [ ]:
step_back_evaluator.get_summary()

In [ ]:
ask_handpicked_questions(step_back_rag)

### Conclusion

# Experiment 4: HyDE approach

The HyDE approach is a novel method that combines human and AI expertise to generate high-quality answers. Originally proposed by Gao et al. (2022) in the paper ["Precise Zero-Shot Dense Retrieval without Relevance Label"](https://arxiv.org/abs/2212.10496), the HyDE pipeline leverages the innovative concept of Hypothetical Document Embeddings (HyDE).

In the HyDE approach, an instruction-following language model, such as our Azure Model using the right prompting, is used to generate a hypothetical document in response to a query. This document is designed to capture the essential relevance patterns from the input question potentially containing inaccuracies or hypothetical details. Following this, the document is used to retrieve relevant documents from the vector store, which are then passed to the LLM model for answer generation.

This two-step process allows the HyDE model to ground the generated hypothetical content in actual data, effectively filtering out incorrect or irrelevant details through the embedding's bottleneck..

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system = """You are an expert about the Clean Technology Sector.
            Answer the user question as best you can. Answer as though you were writing a tutorial that addressed the user question."""

hyde_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

gen_hypothetical_doc = (
        hyde_prompt
        | azure_model
        | StrOutputParser()
)

In [ ]:
hyde_retrieval_chain = (gen_hypothetical_doc
                        | base_retriever
                        )

hyde_rag = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": hyde_retrieval_chain
    }
).assign(answer=base_rag_chain)

hyde_rag.invoke("What is the company's goal in reducing gas flaring?")

### Results

In [ ]:
hyde_evaluator = Evaluator(name="HyDE",
                           cache_results=USE_CACHE,
                           rag_chain=hyde_rag,
                           llm_model=azure_model,
                           embeddings=bge_embeddings)

hyde_evaluator.evaluate(df_eval)
hyde_evaluator.plot_summary()

In [ ]:
hyde_evaluator.get_summary()

In [ ]:
ask_handpicked_questions(hyde_rag)

### Conclusion

# Experiment 5: Contextual Compression


In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

chain_extractor_compressor = LLMChainExtractor.from_llm(azure_model)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=chain_extractor_compressor, base_retriever=base_retriever
)

base_rag_chain_compression = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | ChatPromptTemplate.from_template(base_rag_prompt)
        | azure_model
        | StrOutputParser()
)

base_rag_compression = RunnableParallel(
    {
        "context": compression_retriever,
        "question": RunnablePassthrough()
    }
).assign(answer=base_rag_chain_compression)

base_rag_compression.invoke("Is the company aiming to reduce gas flaring?")

### Results

In [ ]:
base_compression_evaluator = Evaluator(name="Contextual Compression",
                                       cache_results=USE_CACHE,
                                       rag_chain=base_rag_compression,
                                       llm_model=azure_model,
                                       embeddings=bge_embeddings)

base_compression_evaluator.evaluate(df_eval)
base_compression_evaluator.plot_summary()

In [ ]:
base_compression_evaluator.get_summary()

In [ ]:
base_evaluator.get_summary()

In [ ]:
ask_handpicked_questions(base_rag_compression)

### Conclusion